In [25]:
import os
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd


In [26]:
user = "aldi"

In [27]:
# Path to the ChromeDriver
chrome_driver_path = f"C:/Users/{user}/Downloads/chromedriver.exe"

In [28]:
output_folder = f"C:/Users/{user}/Documents/GitHub/tennis-homophily/data/atp"


In [29]:
url = "https://www.atptour.com/en/rankings/doubles?rankRange=1-5000"

In [30]:
%run functions.ipynb

In [31]:
# Configure ChromeOptions for headless mode
options = Options()
#options.add_argument("--headless")
# add options to diagnose errors
options.add_argument("--verbose")
options.add_argument("--log-path=chromedriver.log")


service = Service(chrome_driver_path)

In [33]:
try:
    all_data = []
    dates = []  # List to store corresponding dates

    with webdriver.Chrome(service=service) as driver1:
        driver1.get(url)
        driver1.minimize_window()
        WebDriverWait(driver1, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "mega-table")))
        date_dropdown_ul = WebDriverWait(driver1, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul[data-value='rankDate']"))
        )
        date_options = date_dropdown_ul.find_elements(By.TAG_NAME, "li")

        date_values = []
        for option in date_options:
            date_value = option.get_attribute("data-value")
            if date_value is None:
                continue
            try:
                year = int(date_value[:4])
            except ValueError:
                continue
            if year > 2022:
                date_values.append(date_value)

    filtered_dates = get_highest_dates(date_values)
    print("Filtered Dates:", filtered_dates)

#     filtered_dates = [filtered_dates[-1]]

    for date in filtered_dates:
        with webdriver.Chrome(service=service, options = options) as driver2:
            complete_url = f"https://www.atptour.com/en/rankings/doubles?rankRange=1-5000&rankDate={date}"
            print("Accessing URL for date:", date)  # Print the URL being accessed
            driver2.get(complete_url)
            driver2.minimize_window()
            WebDriverWait(driver2, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "mega-table")))  # Wait for the table to be present
            time.sleep(5)

            page_source = driver2.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            rankings_table = soup.find("table", {"class": "mega-table"})
            if rankings_table is None:
                print(f"Table not found for date: {date}")
                continue  # Skip to the next date if the table is not found

            rows = rankings_table.find_all("tr")[1:25] # Process only the first n rows


            for index, row in enumerate (rows, start = 1):
                rank = row.find("td", {"class": "rank-cell"}).text.strip()
                player_name = row.find("td", {"class": "player-cell"}).text.strip()
                age = row.find("td", {"class": "age-cell"}).text.strip()
                points = row.find("td", {"class": "points-cell"}).text.strip()
                tournaments_played = row.find("td", {"class": "tourn-cell"}).text.strip()
                player_profile_link = row.find("a", href=True)["href"]
                print(f"Processing row {index}: Rank {rank}, Player {player_name}")  # Print the current row being processed


                with webdriver.Chrome(service=service, options = options) as driver3:
                    driver3.get(f"https://www.atptour.com{player_profile_link}")
                    driver3.minimize_window()
                    time.sleep(5)
                    player_page_source = driver3.page_source
                    soup_player = BeautifulSoup(player_page_source, "html.parser")

                    birthplace_div = soup_player.find("div", class_="table-value")
                    birthplace = birthplace_div.get_text(strip=True)
                    city_birthplace, _, country_birthplace = birthplace.partition(",")

                    turned_pro_label_div = soup_player.find("div", class_="table-big-label", text="Turned Pro")
                    year_pro_div = turned_pro_label_div.find_next_sibling("div", class_="table-big-value")
                    year_pro = year_pro_div.get_text(strip=True)

                    weight_span = soup_player.select_one("span.table-weight-kg-wrapper")
                    weight_kg = weight_span.get_text(strip=True).strip("()") if weight_span else ""

                    height_span = soup_player.select_one("span.table-height-cm-wrapper")
                    height_cm = height_span.get_text(strip=True).strip("()") if height_span else ""

                    hand_backhand_div = soup_player.find_all("div", class_="table-value")[1]
                    hand, backhand = [item.strip() for item in hand_backhand_div.get_text(strip=True).split(",")]

                    coach_div = soup_player.find_all("div", class_="table-value")[2]
                    coaches = coach_div.get_text(strip=True).split(", ")
                    coach1 = coaches[0]
                    coach2 = coaches[1] if len(coaches) > 1 else ""

                    all_data.append([rank, player_name, age, points, tournaments_played, year_pro, weight_kg, height_cm, city_birthplace, country_birthplace, hand, backhand, coach1, coach2])
                    dates.append(date)  # Append date for each row

    df = pd.DataFrame(all_data, columns=["Rank", "Player", "Age", "Points", "Tournaments Played", "Year Turned Pro", "Weight (kg)", "Height (cm)", "City of Birthplace", "Country of Birthplace", "Hand", "Backhand", "Coach1", "Coach2"])
    df['Date'] = dates  # Add 'Date' column to the DataFrame

    output_excel_filename = os.path.join(output_folder, "atp_doubles.xlsx")
    df.to_excel(output_excel_filename, index=False)

    print(f"All data saved to {output_excel_filename}")

finally:
    service.stop()


Filtered Dates: ['2023-12-11', '2023-11-27', '2023-10-30', '2023-09-25', '2023-08-28', '2023-07-31', '2023-06-26', '2023-05-29', '2023-04-24', '2023-03-20', '2023-02-27', '2023-01-30']
Accessing URL for date: 2023-12-11
Processing row 1: Rank 1, Player Austin Krajicek
Processing row 2: Rank 2, Player Ivan Dodig
Processing row 3: Rank 3, Player Rohan Bopanna
Processing row 4: Rank 4, Player Matthew Ebden
Processing row 5: Rank 5, Player Horacio Zeballos
Processing row 6: Rank 6, Player Rajeev Ram
Processing row 7: Rank 7, Player Joe Salisbury
Processing row 8: Rank 8, Player Wesley Koolhof
Processing row 9: Rank 9, Player Neal Skupski
Processing row 10: Rank 10, Player Marcel Granollers
Processing row 11: Rank 11T, Player Santiago Gonzalez
Processing row 12: Rank 11T, Player Edouard Roger-Vasselin
Processing row 13: Rank 13T, Player Maximo Gonzalez
Processing row 14: Rank 13T, Player Andres Molteni
Processing row 15: Rank 15, Player Hugo Nys
Processing row 16: Rank 16T, Player Jamie Mur

Processing row 16: Rank 16, Player Matthew Ebden
Processing row 17: Rank 17, Player Kevin Krawietz
Processing row 18: Rank 18, Player Michael Venus
Processing row 19: Rank 19, Player Andreas Mies
Processing row 20: Rank 20, Player Fabrice Martin
Processing row 21: Rank 21, Player Andres Molteni
Processing row 22: Rank 22T, Player Nikola Mektic
Processing row 23: Rank 22T, Player Mate Pavic
Processing row 24: Rank 24, Player Sander Gille
Accessing URL for date: 2023-05-29
Processing row 1: Rank 1T, Player Wesley Koolhof
Processing row 2: Rank 1T, Player Neal Skupski
Processing row 3: Rank 3, Player Rajeev Ram
Processing row 4: Rank 4, Player Joe Salisbury
Processing row 5: Rank 5, Player Austin Krajicek
Processing row 6: Rank 6T, Player Marcelo Arevalo
Processing row 7: Rank 6T, Player Jean-Julien Rojer
Processing row 8: Rank 8, Player Ivan Dodig
Processing row 9: Rank 9, Player Rohan Bopanna
Processing row 10: Rank 10T, Player Lloyd Glasspool
Processing row 11: Rank 10T, Player Harri H

In [ ]:
# shut down operating system after 360 seconds (6 minutes)
#os.system("shutdown /s /t 360")
